In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
df=pd.read_csv("train.csv")

In [3]:
y = df['target'] 
X = df.drop(["ID_code", "target"], axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
models = [
    MLPClassifier(),
        
    RandomForestClassifier(),
    
    GaussianNB()
]

In [6]:
S_train, S_test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [MLPClassifier]


C:\Users\Ayushi.Goel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


    fold  0:  [0.89712500]
    fold  1:  [0.89172500]
    fold  2:  [0.89690000]
    fold  3:  [0.89730000]
    ----
    MEAN:     [0.89576250] + [0.00233536]
    FULL:     [0.89576250]

model  1:     [RandomForestClassifier]


C:\Users\Ayushi.Goel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


    fold  0:  [0.89992500]
    fold  1:  [0.89972500]
    fold  2:  [0.89980000]
    fold  3:  [0.89990000]
    ----
    MEAN:     [0.89983750] + [0.00008004]
    FULL:     [0.89983750]

model  2:     [GaussianNB]
    fold  0:  [0.92075000]
    fold  1:  [0.91980000]
    fold  2:  [0.92190000]
    fold  3:  [0.92205000]
    ----
    MEAN:     [0.92112500] + [0.00091549]
    FULL:     [0.92112500]



In [9]:
import lightgbm as lgb

param = {
        'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1}

d_train = lgb.Dataset(S_train, label=y_train)
model = lgb.train(param, d_train, 100)

y_pred = model.predict(S_test)
print('Final prediction score: [%.8f]' % roc_auc_score(y_test, y_pred))

Final prediction score: [0.68212470]


In [19]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
    
model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
print('Final prediction score: [%.8f]' % accuracy_score(S_test, y_pred))

C:\Users\Ayushi.Goel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [14]:
S_test

array([[0, 0, 0],
       [0, 0, 0],
       [1, 0, 1],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [15]:
S_train

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [16]:
y_train

127478    0
155552    1
75475     0
186114    0
93717     0
118452    0
77896     0
93809     0
155292    0
136594    0
191315    0
36154     1
132074    0
24144     0
117948    0
178501    0
31805     0
198796    0
185056    1
175320    1
56068     0
22418     0
143487    0
157360    0
43674     0
174720    0
63393     0
191225    0
110534    0
34806     0
         ..
83966     0
7877      0
168691    0
70608     0
67699     0
169876    0
137993    0
82457     0
101201    0
108631    0
150055    0
163302    1
17089     0
80186     0
52620     0
170584    0
48600     0
112420    1
186098    0
86293     1
122579    0
41993     0
97639     0
95939     0
152315    0
176963    1
117952    0
173685    0
43567     0
199340    0
Name: target, Length: 160000, dtype: int64